# Deep Learning: Ex.10 - RNN

Submitted by: [... **your name and ID** ...]


In [ ]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input
from tensorflow.keras.layers import SimpleRNN, LSTM # <--- recurrent layers

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
from seaborn import heatmap

print(tf.__version__)

***
### 1. Preprocess the text corpus

(if you are using google colab, remember to upload the corpus file first..)

In [ ]:
f = open('war_and_peace.txt','r') # open the corpus file

text = f.read().lower()  # read file and convert to lower-case letters

print('len(text) = ',len(text))

print(text[:400]) # print the first 400 characters..

- generate training sequences of `T=20` characters, by sampling the text corpus with a stride of 5 characters (i.e., each sequences starts 5 chars after the begining of the last sequences).

- generate a matching list, holding the `next_char` for each of your sequences.

- how many sequences did you extract in total? `N = ?`

- convert the sequences into a 1-hot representation, suitable for our model trainig:

`X.shape = (N, T, len(chars))`

`Y.shape = (N, len(chars))`

In [ ]:
    ###########################
    ###  your code here...  ###
    ###########################

***
### 2. LTSM Model

- Build an `LTSM` model with 128 (hidden)-units that accepts the input sequences. Add a `Dense` layer on top of it, with `len(chars)` softmax units.

- Train the model for only 1 epoch (use: `RMSprop` and batch size of 128).



In [ ]:
    ###########################
    ###  your code here...  ###
    ###########################

### 3. Model predictions


- use the senternce `the meaning of life ` as an input to the model (convert it to 1-hot first..),

- plot the model's output as a probability distribution over the list of chars.

- sample a single char from that distribution, and add it to the generated sentence.

- update the 1-hot buffer, and continue the process for 99 more letters (using a loop).

- print the resulting sentence.

In [ ]:
    ###########################
    ###  your code here...  ###
    ###########################

***
### 3. Fit your model

- Fit your model a bit more (try 10-20 epochs), and regenerate a new `N=100` sentence sample. Does it get any better?

- If you wish, you can try to train the model further, or you can try using a different corpus (dataset) for the training (you can even try a text in hebrew). Be creative ;)

In [ ]:
    ###########################
    ###  your code here...  ###
    ###########################

***
## Good Luck!